# Lecture 11 - EEP 118 Spring 2023¶ 
Nike Zion Williamsons' shoe explosion


This is the notebook for Lecture 11 where we will estimate the effect of a Nike show destruction during an NCAA basketball game on Nike prices, before and after the shoe falls apart on national television.

Can we reject the null hypothesis that Zion’s shoe explosion was not correlated with a change in Nike Stock prices at 5 % significance level, or at 1 % significance? 

Research Strategy: Collect Data on hourly Nike Stock Prices  on Feb 20th (pre shoe explosion) and on Feb 21 (next day after shoe explosion) of 2019



To run, hit the `i>|Run` button on top middle bar and keep hitting and it will run line by line,

OR

To run a line that starts with In [ ]:  highlight the content and hit CONTROL ENTER at same time

In [ ]:
# Load the 'pacman' package
library(pacman) 
#packages to use load them now using the pacman "manager"
p_load(dplyr, haven, readr) 
#Another great feature of p_load(): if you try to load a package that is not installed on your machine, p_load() install the package for you, rather than throwing an error. For instance, let’s install and load one final package named ggplot2. 
p_load(ggplot2)
pacman::p_load(lfe, lmtest, haven, sandwich, tidyverse)
# lfe for running fixed effects regression
# lmtest for displaying robust SE in output table
# haven for loading in dta files

#set scientific display off, thank you Roy
options(scipen=999)


In [ ]:
#-------------------------------------------
#1. Read in data and see the top rows to see column names etc
#-------------------------------------------
#read in a Stata dataset
StockData <- read_dta("Lecture11_stockPricesn.dta")
head(StockData)

In [ ]:
##Renaming first four columns columns
colnames(StockData) <- c("Hour", "Nike", "Adidas", "UnderArmour")

In [ ]:
##Defining Post "Shoe Explodes" Dummy
StockData$PostExplosion <- as.numeric(StockData$Hour >= 9)
#see the bottom rows of the dataset
tail(StockData)

scatter plot for nike stock prices with explosion marked

In [ ]:
NikePlot <- ggplot(StockData, aes(x = Hour, y = Nike))
#NikePlot <- NikePlot + geom_smooth(se=FALSE) + geom_point() + geom_line(color="black") + geom_vline(xintercept=8.5, colour="Red")
NikePlot <- NikePlot + geom_point() + geom_line(color="black") + geom_vline(xintercept=8.5, colour="Red")
NikePlot <- NikePlot + theme_bw() + theme(plot.title = element_text(hjust = 0.5)) + 
  scale_x_continuous(breaks=(seq(1,16,4)), labels=c("9am", "1pm", "9am", "1pm"))
NikePlot <- NikePlot + xlab("Hour") + ylab("Stock Price") + ggtitle("Nike Stock Price: February 20-21, 2019")
NikePlot

Summary Stats

In [ ]:
#one way
library(psych)
describeBy(StockData$Nike, StockData$PostExplosion)
#library(doBy)
#summaryBy(Nike ~ PostExplosion, data = StockData,
#          FUN = function(x) { c(m = mean(x), s = sd(x)) } )
# produces mpg.m wt.m mpg.s wt.s for each
#  level of PostExplosion

Regression of stock price on hourly trend and explosion indicator

In [ ]:
#regression
regLecture11 <- lm(Nike ~ Hour+PostExplosion,StockData)
#show output
summary(regLecture11)

What do we see above? Interpretation, controlling for an hourly trend (Hour),  the explosion is correlated with a significant drop of Nike Stock prices (coefficient on post explosion is -1.27, and is statistically significant from zero, at 5%, 1%, 0.001 % signif the p value is 0.0000244=2.44e-05.

If you want to know if -1.27 dollars is a big drop, compare it to the baseline average of 84.61 (the constant estimate).

So  -1.27/84.61 is about - 0.015, or   -1.5%  drop



How to get estimated parameters bata_hat b0_hat, b1_hat, b2_hat and also Generate Predicted stock price of nike using b0 and b1 and b2 estimates of the regression you estimated

In [ ]:
#how to extract the coeff from the regression above?
#R stores that in recLecture11
#as a variable called regLecture11$coefficients

#all of them into beta_hat vector:
beta_hat<-regLecture11$coefficients
beta_hat

#estimate of a constant is the first one in the coefficients
regLecture11$coefficients[1]

#estimate of explosion parameter is the third one in the coefficients:
regLecture11$coefficients[3]

#and it also has the fitted values and add them as an additional column to the dataframe of data
StockData$Yhat<-regLecture11$fitted.values

#create predictions as a separate column of data called Yhat
Yhat<-regLecture11$fitted.values
#or Nikehat
Nikehat<-regLecture11$fitted.values

Another way to get predictions, multiply coeff and x's 

In [ ]:
StockData$Nikeprediction<- regLecture11$coefficients[1]+ regLecture11$coefficients[2]*StockData$Hour+ regLecture11$coefficients[3]*StockData$PostExplosion
#look at the column Nikeprediction in StockData frame. It is equal to Yhat defined above

Generate the Log of Nike stock price by hour from the original data and add as an additional column to the dataframe called StockData

In [ ]:
#generate log
StockData$NikeLog<-log(StockData$Nike)

We can get and interpret the regression estimates of the log price regression now.  In fact, that way we get the percent change due to the explosion directly from the coefficient of the postExplosion variable,  from the regression of log prices on a constant, hour, and post explosion

In [ ]:
#regression in logs
regLecture11Log <- lm(NikeLog ~ Hour+PostExplosion,StockData)
#show output
summary(regLecture11Log)

What do we see above? Interpretation, controlling for an hourly trend (Hour), the explosion is correlated with a significant drop of log of Nike Stock prices (coefficient on post explosion is -0.015, and is statistically significant from zero, at 5%, 1%, 0.001 % signif the p value is 0.0000248=2.48e-05. A change in the log of -0.015 in the change in the Nike price of - 1.5%

beta_hat post explosion=-0.015 corresponds to a -1.5% drop in the Nike price.

THIS is a before and AFTER analysis, we can say if we find statistical evidence 
in favor of explosion being correlated with a change in Nike stock or not, 
all else constant. But we cannot say it CAUSED the change in price.

Later in class, where we are going towards is CAUSALITY!

Research Strategy: Collect Data on hourly Nike and UA and Adidas Stock Prices  on Feb 20 and 21 

That is, get data also for a control group that captures anything else that could have happened to stock prices for sports companies from 20 and 21 of Feb that had nothing to do with the explosion, e.g. get stock prices for Adidas and Under Armour.

FIRST : Check if Adidas and Under Armour, or average of (UA and Adidas),  stock prices were trending similarly to Nike stock prices before the explosion

Then we have a good counterfactual, Of what would have happened in Feb 21 relative to Feb 20 to Nike if there was no explosion (using the change in average (UA and Adidas) stock prices as that control change)

Then the causal effect of the explosion on Nike on Feb 21 is change in Nike MINUS change in average(UA and Adidas). We call this Difference in Differences. 

See the youtube of (NikeLab folder).

I will return at end of semester to do the rest with you all, and see in the next slide a blog post  on my complete Econometric analysis  Nike Lab. 

With Scott Kaplan, who is starting this Fall as an Assistant Professor in Economics at the U. S. Naval academy. He graduated from EEP, then will graduate this year with a PhD in ARE 

https://www.scottkaplan.org/post/the-explosion-heard-around-the-college-basketball-world

The video of this Lecture11 portion – Nike Lab, is here on Youtube, by yours truly
https://youtu.be/eWE_vEEZhS0


